# Recap

In [1]:
%%writefile main.kv

#:import random random

<Widget>:
    canvas:
        Color:
            rgba: .5,.5,.5,.5
        Line:
            rectangle: self.x, self.y, self.width, self.height
            width: 2
                
<DebugLabel@Button>:
    size: self.parent.size
    pos: self.parent.pos
    background_color: random.random(), random.random(), random.random(), 0.6
    text: 'debuglabel'


Overwriting main.kv


In [2]:
%%writefile main.py

from kivy.app import App
from selectiontool import SelectionTool


class MainApp(App):
    title = 'Image Selection Tool'

    def build(self):
        return SelectionTool()


if __name__ == '__main__':
    MainApp().run()

Overwriting main.py


In [3]:
%%writefile selectiontool.py

from kivy.lang import Builder
from kivy.uix.boxlayout import BoxLayout

Builder.load_file('selectiontool.kv')

class SelectionTool(BoxLayout):
    pass


Overwriting selectiontool.py


In [4]:
%%writefile selectiontool.kv

<SelectionTool>:
    orientation: "vertical"

    BoxLayout:
        orientation: "horizontal"
            
        DebugLabel:
            text: ""
            size_hint_x: 4
        DebugLabel:
            text: "Book"
        DebugLabel:
            text: "Page"

    DebugLabel:
        text: "Delete"

    BoxLayout:
        size_hint_y: 16
        orientation: "horizontal"
        DebugLabel:
            text: "Words"
            size_hint_x: 0.2
        DebugLabel:
            text: "Image"
            size_hint_x: 0.8


Overwriting selectiontool.kv


# Tool layout

Let's replace the mockup `selectiontool.kv` with the real one.

In [5]:
%%writefile selectiontool.kv

<SelectionTool>:
    orientation: "vertical"

    book_selector: _book_selector
    page_selector: _page_selector
    image_pane: _image_pane
    word_list: _word_list

    book_id: self.book_selector.text
    page: self.page_selector.text

    BoxLayout:
        orientation: 'horizontal'

        Widget:
            size_hint_x: 4

        Spinner:
            id: _book_selector

        Spinner:
            id: _page_selector

    Button:
        text: "Delete Last Rectangle"
        on_press: root.image_pane.delete_last_rectangle()

    BoxLayout:
        orientation: 'horizontal'
        size_hint_y: 16
        BoxLayout:
            id: _word_list
            size_hint_x: 0.2
            orientation: 'vertical'

        ImagePane:
            id: _image_pane
            size_hint_x: 0.8
            source: ''


Overwriting selectiontool.kv


Notice that we replaced the "Book" and "Page" `DebugLabels` with `Spinners`. These are drop down selectors.

"Words" is now an empty `BoxLayout`. We will add to it programmatically.

The "Image" `DebugLabel` has been replaced by an `ImagePane`. This is our own class, which is an `Image` that will have additional properties and methods (in particular, it will draw a rectangle when you drag on it).

Finally, what are these ids? We have given the book spinner the `id` "_book_selector". Then at the top we have
```
book_selector: _book_selector
```
So in the kv file or in the `SelectionTool` python code, we will be about to reference the book spinner with `self.book_selector`. For example, we set
```
    book_id: self.book_selector.text
```

Finally, let's define `ImagePane` (we'll leave it empty for now).

In [6]:
%%writefile imagepane.py

from kivy.uix.image import Image
from kivy.lang import Builder
from kivy.app import App


class ImagePane(Image):
    pass


Overwriting imagepane.py


In [7]:
%%writefile selectiontool.py

from kivy.lang import Builder
from kivy.uix.boxlayout import BoxLayout
from imagepane import ImagePane


Builder.load_file('selectiontool.kv')

class SelectionTool(BoxLayout):
    pass



Overwriting selectiontool.py


In [8]:
#!python main.py

<img src="Images/rough_code_mockup.png"/>

The Book and Page spinners are empty, the image is a small white rectangle, and the Delete button crashes the app. But the layout is good. Progress!

# Setting the spinners

The spinners are empty. We have to give them a list of possible `values`, and set which is initially selected. The `Example_Data` directory contains directories for the books.

In [9]:
!ls Example_Data/

0001 0257


In [10]:
!ls Example_Data/0001/

01.jpg        04.jpg        07.jpg        10.jpg
02.jpg        05.jpg        08.jpg        11.jpg
03.jpg        06.jpg        09.jpg        word_list.txt


The jpegs are the page images, while `word_list.txt` contains words we want to find images for in the book.

In [11]:
%%writefile selectiontool.py

import os
import glob

from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.properties import StringProperty
from kivy.lang import Builder

from imagepane import ImagePane

Builder.load_file('selectiontool.kv')

class SelectionTool(BoxLayout):
    library_directory = 'Example_Data'

    book_id = StringProperty()
    page = StringProperty()

    def __init__(self):
        super(SelectionTool, self).__init__()
        book_pattern = os.path.join(self.library_directory, '[0-9]' * 4)
        self.book_selector.values = [os.path.basename(s) for s in glob.glob(book_pattern)]
        self.book_selector.text = self.book_selector.values[0] if self.book_selector.values else 'No Books'


Overwriting selectiontool.py


In [12]:
#!python main.py

<img src="Images/book_spinner_set.png"/>

### An aside: Kivy properties

A _Kivy property_ is a normal property with an event-driven twist: 
* when a Kivy property changes, it generates an event that any class can pick up and act on
* Kivy properties can be tied together, so that when one changes, the other automatically changes.


### Back to the code

The `page_selector` spinner contains the page numbers for the book that is selected. What happens when we pick a book? the `book_selector` spinner gets a new value for `text`. In `selectiontool.kv` we set
```
    book_id: self.book_selector.text

```
This makes book_id a kivy property tied to `book_selector.text`. So when we select a new value for the book spinner, we end up with a new value in `book_id`. Since `book_id` is a Kivy property, this fires off an event. The event will automatically call a method named `on_book_id()` if it exists. So let's make one!

In [13]:
%%writefile -a selectiontool.py

    def on_book_id(self, inst=None, value=None):
        image_pattern = os.path.join(self.library_directory, self.book_id, '*.jpg')
        self.page_selector.values = [os.path.basename(s)[:-4] for s in glob.glob(image_pattern)]
        self.page_selector.text = self.page_selector.values[0] if self.page_selector.values else 'No Pages'


Appending to selectiontool.py


In [14]:
#!python main.py

<img src="Images/page_spinner_set.png"/>

# Set the word list

We also need to load the word list when we change books.

In [15]:
%%writefile -a selectiontool.py

    def on_book_id(self, inst=None, value=None):
        image_pattern = os.path.join(self.library_directory, self.book_id, '*.jpg')
        self.page_selector.values = [os.path.basename(s)[:-4] for s in glob.glob(image_pattern)]
        self.page_selector.text = self.page_selector.values[0] if self.page_selector.values else 'No Images'

        self.word_list.clear_widgets()
        with open(os.path.join(self.library_directory, self.book_id, 'word_list.txt')) as fp:
            for word in sorted(fp.readlines()):
                self.word_list.add_widget(Label(text=word.strip()))


Appending to selectiontool.py


Notice that here we are manually changing the widgets in the word list BoxLayout. `clear_widgets()` removes all widgets from the BoxLayout, while `add_widget()` adds a widget. As widgets are added the BoxLayout automatically resizes and repositions them so they all fit.

In [16]:
#!python main.py

<img src="Images/word_list_set.png"/>

# Set the image

So, when we choose a new page we want to load the image for that page onto the screen. Sounds like we need an `on_page()` method!

In [17]:
%%writefile -a selectiontool.py

    def on_page(self, *_):
        page_filename = self.page + '.jpg'
        self.image_pane.source = os.path.join(self.library_directory, self.book_id, page_filename)
        

Appending to selectiontool.py


In [18]:
#!python main.py

<img src="Images/image_set.png"/>

Well, the image is there, but we'd like it to fill the image box.

In [19]:
%%writefile imagepane.py

from kivy.uix.image import Image
from kivy.lang import Builder
from kivy.app import App

Builder.load_string("""
<ImagePane>:
    allow_stretch: True
""")

class ImagePane(Image):
    pass


Overwriting imagepane.py


Note that here we've using kv language from a string rather than a file.

In [20]:
#!python main.py

<img src="Images/image_stretched.png"/>

This is good... Except that if we are on page 1 then selecting a different book changes the word list but not the image!

# Exercise

Why? Fix this.